In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from os import system
import os
import keras
import util
from __future__ import print_function

In [2]:
import xml.etree.ElementTree as ET
import classification_starter as start
import os
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import numpy as np
from scipy import sparse

import util

In [37]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from gensim import corpora, models, similarities
import gensim

In [21]:
labels = []
with open('t_train_starter.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        labels.append(row)

sentences = []
sent_lengths = []
with open('sentences.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        sent_lengths.append(len(row))
        sentences.append(row)

sentences_valid = []
sent_lengths_valid = []
with open('sentences_valid.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        sent_lengths_valid.append(len(row))
        sentences_valid.append(row)

In [61]:
y_train = np.array(labels[0][:200]).astype(int).T
y_valid = np.array(labels[0][200:300]).astype(int).T

In [24]:
# Set values for various parameters
num_features = 500    # Word vector dimensionality                      
min_word_count = 15   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 50          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [29]:
# Initialize and train the model (this will take some time)
from gensim.models import word2vec


model = word2vec.Word2Vec(sentences, workers=num_workers, 
                          size=num_features, min_count=min_word_count, 
                          window=context, sample=downsampling)

/Users/hikarisorensen/anaconda/lib/python2.7/site-packages/gensim/__init__.py:10: UserWarning: Module gensim was already imported from /Users/hikarisorensen/anaconda/lib/python2.7/site-packages/gensim/__init__.pyc, but /Users/hikarisorensen/anaconda/lib/python2.7/site-packages/gensim-1.0.1-py2.7-macosx-10.7-x86_64.egg is being added to sys.path
  __version__ = __import__('pkg_resources').get_distribution('gensim').version
/Users/hikarisorensen/anaconda/lib/python2.7/site-packages/gensim/__init__.py:10: UserWarning: Module smart_open was already imported from /Users/hikarisorensen/anaconda/lib/python2.7/site-packages/smart_open/__init__.pyc, but /Users/hikarisorensen/anaconda/lib/python2.7/site-packages/smart_open-1.4.0-py2.7.egg is being added to sys.path
  __version__ = __import__('pkg_resources').get_distribution('gensim').version


In [34]:
model.wv.syn0.shape

(5732, 500)

In [38]:
# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 10

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [58]:
# For the first 10 clusters
for cluster in xrange(0,2):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words

SyntaxError: invalid syntax (<ipython-input-58-63f4c6389cd1>, line 12)

['A1', 'BF', 'B1', '04', '6A', '7D']

In [51]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [52]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (len(sentences), num_clusters), \
    dtype="float32" )

# Transform the training set into bags of centroids
counter = 0
for sentence in sentences:
    train_centroids[counter] = create_bag_of_centroids( sentence, \
        word_centroid_map )
    counter += 1


valid_centroids = np.zeros(( len(sentences_valid), num_clusters), \
    dtype="float32" )

counter = 0
for sentence in sentences_valid:
    valid_centroids[counter] = create_bag_of_centroids( sentence, \
        word_centroid_map )
    counter += 1

In [60]:
train_centroids = train_centroids.astype(int)
valid_centroids = valid_centroids.astype(int)

In [62]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM


max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32


In [63]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(train_centroids, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(valid_centroids, y_valid))
score, acc = model.evaluate(valid_centroids, y_valid,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...
Train on 200 samples, validate on 100 samples
Epoch 1/15
 32/200 [===>..........................] - ETA: 8s - loss: 0.5576 - acc: 0.0000e+00

IndexError: index 37175 is out of bounds for size 20000
Apply node that caused the error: AdvancedSubtensor1(if{inplace}.0, Reshape{1}.0)
Toposort index: 93
Inputs types: [TensorType(float32, matrix), TensorType(int32, vector)]
Inputs shapes: [(20000, 128), (18336,)]
Inputs strides: [(512, 4), (4,)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Reshape{3}(AdvancedSubtensor1.0, MakeVector{dtype='int64'}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "<ipython-input-63-80e3ef35c667>", line 3, in <module>
    model.add(Embedding(max_features, 128, dropout=0.2))
  File "/Users/hikarisorensen/anaconda/lib/python2.7/site-packages/keras/models.py", line 299, in add
    layer.create_input_layer(batch_input_shape, input_dtype)
  File "/Users/hikarisorensen/anaconda/lib/python2.7/site-packages/keras/engine/topology.py", line 401, in create_input_layer
    self(x)
  File "/Users/hikarisorensen/anaconda/lib/python2.7/site-packag

In [65]:
print len(train_centroids)

SyntaxError: invalid syntax (<ipython-input-65-8798ce06ba35>, line 1)

In [ ]:
system("say Finished")

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(32, input_dim=784),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])